# ADD drogue status

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
#import histlib.diagnosis as diag
import histlib.erastar as eras

from dask.distributed import wait
from histlib.cstes import labels, zarr_dir

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    cluster = PBSCluster(cores=7, processes=7, walltime="04:00:00")
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster

    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.70:8787/status,
Dashboard: http://10.148.1.70:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.70:36545,Workers: 0
Dashboard: http://10.148.1.70:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# TEST

In [4]:
# create 
l='gps_Jason-3_2020'
ds_data = xr.open_zarr(os.path.join(zarr_dir,'test', 'gps_Jason-3_2020.zarr')).chunk({'obs':5}).persist()
#ds_data = add_adt_to_ds_data(ds_data)

In [15]:
def drogue_status(ds_data):
    return ds_data.time<ds_data.drifter_drogue_lost_date.mean('site_obs')
    
drogue_status(ds_data)

AttributeError: 'numpy.ndarray' object has no attribute 'date'

In [6]:
ds_data = ds_data.compute()

In [13]:
ds_data

<xarray.Dataset>
Dimensions:                                       (obs: 9, alti_time: 211,
                                                   alti_time_mid: 210,
                                                   box_y: 80, box_x: 120,
                                                   site_obs: 1489)
Coordinates: (12/20)
    alti_lat                                      (obs, alti_time) float64 -3...
    alti_lon                                      (obs, alti_time) float64 13...
  * alti_time                                     (alti_time) int64 0 1 ... 210
    alti_time_                                    (obs, alti_time) datetime64[ns] ...
  * alti_time_mid                                 (alti_time_mid) float64 0.5...
    alti_x                                        (obs, alti_time) float64 -7...
    ...                                            ...
    drifter_x                                     (obs, site_obs) float64 -5....
    drifter_y                                     (obs, site_obs) float64 -3....
    lat                                           (obs) float64 -41.18 ... 30.65
    lon                                           (obs) float64 139.0 ... -24.16
  * obs                                           (obs) int64 0 2 3 4 5 6 7 8 9
    time                                          (obs) datetime64[ns] 2020-0...
Dimensions without coordinates: site_obs
Data variables: (12/68)
    __site_id                                     (obs) object '60753420' ......
    __site_matchup_indice                         (obs) float64 744.0 ... 744.0
    __site_name                                   (obs) object '60753420' ......
    alti___distance                               (obs) float64 9.718e+04 ......
    alti___source                                 (obs) object 'dt_global_j3_...
    alti___source_center_index                    (obs) object '44309' ... '1...
    ...                                            ...
    drifter_typedeath                             (obs, site_obs) float64 1.0...
    drifter_ve                                    (obs, site_obs) float32 0.1...
    drifter_vn                                    (obs, site_obs) float32 0.2...
    drifter_vx                                    (obs, site_obs) float64 -0....
    drifter_vy                                    (obs, site_obs) float64 0.2...
    f                                             (obs) float64 -9.602e-05 .....
Attributes:
    __id:                   CMEMS-L3-SLA-Jason-3__gps
    __time_coverage_end:    2020-05-01T00:00:00
    __time_coverage_start:  2020-01-01T00:00:00

In [11]:
ds_data.time

<xarray.DataArray 'time' (obs: 9)>
array(['2020-01-02T21:00:00.000000000', '2020-01-02T02:00:00.000000000',
       '2020-01-02T12:00:00.000000000', '2020-01-02T05:00:00.000000000',
       '2020-01-02T20:00:00.000000000', '2020-01-02T13:00:00.000000000',
       '2020-01-02T22:00:00.000000000', '2020-01-02T13:00:00.000000000',
       '2020-01-02T05:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    lat      (obs) float64 -41.18 -36.9 -28.8 31.07 33.6 51.25 30.41 52.3 30.65
    lon      (obs) float64 139.0 62.7 59.0 -23.42 ... -59.59 -156.3 -24.16
  * obs      (obs) int64 0 2 3 4 5 6 7 8 9
    time     (obs) datetime64[ns] 2020-01-02T21:00:00 ... 2020-01-02T05:00:00
Attributes:
    long_name:      time of the nearest site data
    standard_name:  time

In [12]:
ds_data.drifter_drogue_lost_date.mean('site_obs')

<xarray.DataArray 'drifter_drogue_lost_date' (obs: 9)>
array(['2015-06-23T00:00:00.000000000', '2014-06-10T00:00:00.000000000',
       '2017-10-23T00:00:00.000000000', '2015-07-17T00:00:00.000000000',
       '2016-10-23T00:00:00.000000000',                           'NaT',
       '2016-07-31T00:00:00.000000000',                           'NaT',
       '2016-01-31T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    lat      (obs) float64 -41.18 -36.9 -28.8 31.07 33.6 51.25 30.41 52.3 30.65
    lon      (obs) float64 139.0 62.7 59.0 -23.42 ... -59.59 -156.3 -24.16
  * obs      (obs) int64 0 2 3 4 5 6 7 8 9
    time     (obs) datetime64[ns] 2020-01-02T21:00:00 ... 2020-01-02T05:00:00

In [16]:
cluster.close()